In [1]:
from nltk.stem import WordNetLemmatizer
#from functools32 import lru_cache
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk

def tokens(sentence):
  return word_tokenize(sentence)

def lemmatize(tokens):
  wnl = WordNetLemmatizer()
  return [wnl.lemmatize(token) for token in tokens]

def postags(tokens):
  return nltk.pos_tag(tokens)

In [2]:
sentence = "My name is Daniel"
word_tokens = tokens(sentence)
print lemmatize(word_tokens)
print postags(word_tokens)

import glob
import re

fname = "../STS-data/STS2013-gold/STS.input.headlines.txt"
with open(fname) as f:
    lines = f.readlines()

Data = []
    
for fname in glob.glob("../STS-data/*201[1-5]*/*input*.txt"):
    print fname
    with open(fname) as f:
        lines = f.readlines()
        for line in lines:
            if len(line.split('\t')) !=2 :
                print fname
                break;
            else:
                Data.append(line.split('\t'))
for fname in glob.glob("../STS-data/*2016*/*input*.txt"):
    print fname
    print re.sub("input","gs",fname)
    with open(fname) as f:
        lines = f.readlines()
        with open(re.sub("input","gs",fname)) as nfile:
            nlines = nfile.readlines()
        #nlines[index]
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=4 :
                #print fname
                break;
            else:
                x = line.split('\t')
                Data.append([x[0],x[1],nlines[index]])
Data[-1]

['My', 'name', 'is', 'Daniel']
[('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Daniel', 'NNP')]
../STS-data/STS2012-gold/STS.input.MSRpar.txt
../STS-data/STS2012-gold/STS.input.MSRvid.txt
../STS-data/STS2012-gold/STS.input.SMTeuroparl.txt
../STS-data/STS2012-gold/STS.input.surprise.OnWN.txt
../STS-data/STS2012-gold/STS.input.surprise.SMTnews.txt
../STS-data/STS2012-train/STS.input.MSRpar.txt
../STS-data/STS2012-train/STS.input.MSRvid.txt
../STS-data/STS2012-train/STS.input.SMTeuroparl.txt
../STS-data/STS2013-gold/STS.input.FNWN.txt
../STS-data/STS2013-gold/STS.input.headlines.txt
../STS-data/STS2013-gold/STS.input.OnWN.txt
../STS-data/STS2014-gold/STS.input.deft-forum.txt
../STS-data/STS2014-gold/STS.input.deft-news.txt
../STS-data/STS2014-gold/STS.input.headlines.txt
../STS-data/STS2014-gold/STS.input.images.txt
../STS-data/STS2014-gold/STS.input.OnWN.txt
../STS-data/STS2014-gold/STS.input.tweet-news.txt
../STS-data/STS2015-gold/STS.input.answers-forums.txt
../STS-data/STS2015-gold/

["Is it inappropriate to spend personal resources for the company's benefit?",
 "Is it ethical/legal to use the University's resources for a personal project?",
 '\n']

In [3]:
Data = []
fname = "../STS-data/STS2013-gold/STS.input.headlines.txt"
with open(fname) as f:
    lines = f.readlines()

    with open(re.sub("input","gs",fname)) as nfile:
        nlines = nfile.readlines()
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=2 :
                #print fname
                break;
            else:
                if(re.search("(\d)((\.)(\d))?", nlines[index])):
                    ;#print nlines[index]
                else:
                    print lines[index]
                    break
                x = line.split('\t')
                Data.append([x[0],x[1], float(re.sub("\n","",nlines[index]))])
                
#print Data

In [4]:
Data = []
fname = "../STS-data/sts2016-english-with-gs-v1.0/STS2016.input.question-question.txt"
with open(fname) as f:
    lines = f.readlines()

    with open(re.sub("input","gs",fname)) as nfile:
        nlines = nfile.readlines()
        #print nlines
        
        for index,line in enumerate(lines):
            if len(line.split('\t')) !=4 :
                #print fname
                break;
            else:
                if(not re.search("(\d)((\.)(\d))?", nlines[index])):
                    continue;#print nlines[index]
                #else:
                    #print lines[index]
                    #continue
                x = line.split('\t')
                Data.append([x[0],x[1], float(re.sub("\n","",nlines[index]))])
                
#print Data

In [69]:
from collections import defaultdict
from collections import Counter
import preprocess as process
import utilities as utility
import math

def ngram_vector_keys(tokens, ngram_size=1):
  vector_keys = []
  for i in range(len(tokens)-(ngram_size-1)):
    vector_keys.append(tuple(tokens[i:i+(ngram_size)]))
    # vector_keys.append(" ".join(tokens[i:i+(ngram_size)]))
  return vector_keys

def containment_coefficienct(sent_1_tokens, sent_2_tokens, ngram_size=1):
  set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
  set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  containment_of_sentence_1_in_2 = float(len(set1.intersection(set2)))/len(set1)
  containment_of_sentence_2_in_1 = float(len(set1.intersection(set2)))/len(set2)
  return containment_of_sentence_1_in_2, containment_of_sentence_2_in_1

def JaccardCoefficient(sent_1_tokens, sent_2_tokens, ngram_size=1):
  set1 = set(ngram_vector_keys(sent_1_tokens, ngram_size))
  set2 = set(ngram_vector_keys(sent_2_tokens, ngram_size))
  #print set1,set2
  return float(len(set1.intersection(set2)))/len(set1.union(set2))

def TFIDF(documents):
  Vocabulary = Counter()
  DocVectors = []
  IDFVector = Counter()
  No_of_Documents = float(len(documents))
  for document in documents:
    tf_single_doc_count = Counter(process.tokens(document))
    Vocabulary+= tf_single_doc_count
    DocVectors.append(tf_single_doc_count)
    IDFVector += Counter(tf_single_doc_count.keys())
  # print IDFVector
  for key in IDFVector.keys():
    IDFVector[key] = math.log(No_of_Documents/(1+IDFVector[key]))
  # print IDFVector
  # IDFVector = defaultdict(lambda:0.0, dict((key,Vocabulary[key]*) for key in c.keys()))
  TFIDFScores = defaultdict(lambda:0.0, dict((key,Vocabulary[key]*IDFVector[key]) for key in Vocabulary.keys()))
  # print TFIDFScores, IDFVector, Vocabulary
  return TFIDFScores, Vocabulary, DocVectors, IDFVector

def DocvectorTFIDF(TFIDFScores, tokens):
  return defaultdict(lambda:0.0, dict((key,TFIDFScores[key] if key in TFIDFScores.keys() else 0.0) for key in tokens))


def cosinesimilarity(document1, document2, TFIDFScores):
  tokens1 = set(process.tokens(document1))
  tokens2 = set(process.tokens(document2))
  vector1 = DocvectorTFIDF(TFIDFScores, tokens1)
  vector2 = DocvectorTFIDF(TFIDFScores, tokens2)
  len_vector_1 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_1
  len_vector_2 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_2
  print utility.dict_dotprod(vector1,vector2)
  cosine_similarity_score = (utility.dict_dotprod(vector1,vector2))/float((len_vector_2*len_vector_1))
  return cosine_similarity_score


def cosinesimilarity_without_TFIDF(document1, document2):
  vector1 = Counter(process.tokens(document1))
  vector2 = Counter(process.tokens(document2))
  print vector1
  print vector2
  len_vector_1 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_1
  len_vector_2 = math.sqrt(sum({k: v**2 for k, v in vector1.items()}.values()))
  print len_vector_2
  cosine_similarity_score = (utility.dict_dotprod(vector1,vector2))/float((len_vector_2*len_vector_1))
  return cosine_similarity_score


In [71]:
TFIDFScores, Vocabulary, DocVectors, IDFVector = TFIDF(["i ike", "i ike"])
TFIDFScores = {"i":1,"ike":1}
v = DocvectorTFIDF(TFIDFScores, process.tokens("i ike"))
#print v
#print utility.dict_dotprod(v,DocvectorTFIDF(TFIDFScores, process.tokens("i tes")))
print cosinesimilarity("i ike", "i tes", {"i":1,"ike":1})
print cosinesimilarity_without_TFIDF("i ike", "i ike")

JaccardCoefficient(["sent_1_tokens","test"], ["sent_2_tokens","tes","Tesd","Tesd","Tesd"], ngram_size=2)
print containment_coefficienct(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=1)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=2)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=3)
print JaccardCoefficient(["a","rose","is","a","rose","is","a","rose"],["a","rose","is","a","flower","which","is","a","rose"],ngram_size=4)

1.41421356237
1.41421356237
1.0
0.5
Counter({'i': 1, 'ike': 1})
Counter({'i': 1, 'ike': 1})
1.41421356237
1.41421356237
1.0
(1.0, 0.6)
0.6
0.5
0.428571428571
0.125
